In [95]:
import sys
sys.path.append('../')
from datetime import datetime
from utils.extract_amplitude_data import start_session
from utils.df_utils import fill_na_columns
actual_date = datetime.now().strftime("%Y-%m-%d")
from invest_per_state.utils.amplitude.api_call import get_rql_data
from invest_per_state.utils.utils import filter_by_date_range
import pandas as pd

## RQLs de Amplitude

In [96]:
#
# !: Se puede sacar el historial apartir de acá

In [97]:
data_rql_amplitude = get_rql_data(actual_date)
data_rql_amplitude

Iniciando solicitud...
Respuesta recibida en 7.11 segundos
Código de estado: 200
Procesando respuesta...
Datos guardados en '2025-04-14-amplitude_data.json'
Datos procesados correctamente
Tiempo total: 8.70 segundos


,publication_code,brand,state,Time,MX RQL Online (By LS)--All Users
0,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-03-15T00:00:00,4.0
1,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-03-16T00:00:00,5.0
2,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-03-17T00:00:00,7.0
3,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-03-18T00:00:00,14.0
4,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-03-19T00:00:00,18.0
...,...,...,...,...,...
100466,MX2454-benelli-vz-125i,Benelli,Guanajuato,2025-04-10T00:00:00,0.0
100467,MX2454-benelli-vz-125i,Benelli,Guanajuato,2025-04-11T00:00:00,0.0
100468,MX2454-benelli-vz-125i,Benelli,Guanajuato,2025-04-12T00:00:00,0.0
100469,MX2454-benelli-vz-125i,Benelli,Guanajuato,2025-04-13T00:00:00,0.0


Filtrar por las fechas de los RQLs a buscar

In [98]:
date_range = ["2025-04-07", "2025-04-08", "2025-04-09", "2025-04-10", "2025-04-11", "2025-04-12", "2025-04-13"]
data_rql_amplitude = filter_by_date_range(data_rql_amplitude, date_range)

Filtrar quitando RQLs vacíos

In [99]:
data_rql_amplitude.loc[:, "MX RQL Online (By LS)--All Users"] = data_rql_amplitude.loc[:, "MX RQL Online (By LS)--All Users"].astype(float)
data_rql_amplitude = data_rql_amplitude[data_rql_amplitude["MX RQL Online (By LS)--All Users"] != 0]
total_rqls = data_rql_amplitude['MX RQL Online (By LS)--All Users'].sum()
display(data_rql_amplitude)
print(f"Total de MX RQL Online (By LS)--All Users: {total_rqls}")

,publication_code,brand,state,Time,MX RQL Online (By LS)--All Users,day
23,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-04-07T00:00:00,32.0,2025-04-07
24,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-04-08T00:00:00,28.0,2025-04-08
25,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-04-09T00:00:00,35.0,2025-04-09
26,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-04-10T00:00:00,28.0,2025-04-10
27,MX329-bajaj-pulsar-n-250,Bajaj,México,2025-04-11T00:00:00,26.0,2025-04-11
...,...,...,...,...,...,...
100122,MX1238-kawasaki-kx-250-2023,Kawasaki,Nuevo León,2025-04-07T00:00:00,1.0,2025-04-07
100218,MX1233-ktm-1290-super-duke-r-2023,KTM,México,2025-04-10T00:00:00,1.0,2025-04-10
100339,MX1917-hero-eco-150-cargo,Hero,Jalisco,2025-04-07T00:00:00,1.0,2025-04-07
100407,MX2621-cf-moto-cforce-1000-touring,CF Moto,Jalisco,2025-04-13T00:00:00,1.0,2025-04-13


Total de MX RQL Online (By LS)--All Users: 4851.0


## Datos de Geolocalización

In [100]:
df_state_per_media = pd.read_csv(r"C:\Users\JTRUJILLO\Documents\Galgo\Scripts\Análisis\rql_inversiones\Geolocalización\MX equivalencias geo regiones - Equivalencias_Medios.csv")

Combinar datos de geolocalización

In [101]:
df_rql_per_state_and_brand_merged = pd.merge(data_rql_amplitude, df_state_per_media, left_on="state", right_on="Amplitude_ips", how = "left")
df_rql_per_state_and_brand_merged.drop(columns = ["Time", "publication_code", "Amplitude_ips"], inplace = True)
df_rql_per_state_and_brand_merged.rename(columns = {"MX RQL Online (By LS)--All Users": "rql"}, inplace = True)
df_rql_per_state_and_brand_merged = df_rql_per_state_and_brand_merged[["day", "state", "brand", "rql", "Meta", "GAds", "TikTok"]]
# df_rql_per_state_and_brand_merged.drop(columns = ["Amplitude_declarado"], inplace = True)

In [102]:
# Crear columnas de año y semana
df_rql_per_state_and_brand_merged['Año'] = pd.to_datetime(df_rql_per_state_and_brand_merged['day']).dt.year
df_rql_per_state_and_brand_merged['Semana'] = pd.to_datetime(df_rql_per_state_and_brand_merged['day']).dt.isocalendar().week
df_rql_per_state_and_brand_merged

,day,state,brand,rql,Meta,GAds,TikTok,Año,Semana
0,2025-04-07,México,Bajaj,32.0,State of Mexico,State of Mexico,Mexico,2025,15
1,2025-04-08,México,Bajaj,28.0,State of Mexico,State of Mexico,Mexico,2025,15
2,2025-04-09,México,Bajaj,35.0,State of Mexico,State of Mexico,Mexico,2025,15
3,2025-04-10,México,Bajaj,28.0,State of Mexico,State of Mexico,Mexico,2025,15
4,2025-04-11,México,Bajaj,26.0,State of Mexico,State of Mexico,Mexico,2025,15
...,...,...,...,...,...,...,...,...,...
2926,2025-04-07,Nuevo León,Kawasaki,1.0,Nuevo León,Nuevo Leon,Nuevo Leon,2025,15
2927,2025-04-10,México,KTM,1.0,State of Mexico,State of Mexico,Mexico,2025,15
2928,2025-04-07,Jalisco,Hero,1.0,Jalisco,Jalisco,Jalisco,2025,15
2929,2025-04-13,Jalisco,CF Moto,1.0,Jalisco,Jalisco,Jalisco,2025,15


In [103]:
print(df_rql_per_state_and_brand_merged["rql"].sum())

4851.0


## RQL por cada una de las marcas

Se agrupan para evitar tener registros duplicados al hacer un combinado

In [104]:
# Agrupar por Marca, Semana y Año, y sumar el costo
rql_resumen_per_state = df_rql_per_state_and_brand_merged.groupby(['state', 'Semana', 'Año'])['rql'].sum().reset_index()
display(rql_resumen_per_state.head(2))
print(rql_resumen_per_state["rql"].sum())

,state,Semana,Año,rql
0,Aguascalientes,15,2025,14.0
1,Baja California,15,2025,9.0


4851.0


In [105]:
rql_resumen_per_brand = df_rql_per_state_and_brand_merged.groupby(['brand', 'Semana', 'Año'])['rql'].sum().reset_index()
display(rql_resumen_per_brand.head(2))
print(rql_resumen_per_brand["rql"].sum())

,brand,Semana,Año,rql
0,Bajaj,15,2025,1927.0
1,Benelli,15,2025,63.0


4851.0


In [106]:
from datetime import datetime
actual_date = datetime.now().strftime("%Y%m%d")
print(actual_date)

20250414


In [107]:
rql_resumen_per_state.to_csv(f"outputs/RQLs Amplitude/{actual_date}-rql_resumen_per_state.csv", index=False)
rql_resumen_per_brand.to_csv(f"outputs/RQLs Amplitude/{actual_date}-rql_resumen_per_brand.csv", index=False)

## Verificación de cantidad de RQLs generados para una semana

In [13]:
resumen_rqls_por_marca_week = resumen_rqls_por_marca[resumen_rqls_por_marca["Semana"] == 14]
resumen_rqls_por_marca_week["brand"].unique()

array(['Bajaj', 'Dinamo', 'Honda', 'Vento', 'Yamaha', 'Suzuki', 'Hero',
       'Italika', 'TVS', 'Zmoto', 'Kawasaki', 'CF Moto', 'CFLITE', 'KTM',
       'MB', 'Benelli', 'Treck', 'Kymco', 'Royal Enfield', 'Husqvarna',
       'Jiajue', 'Zontes'], dtype=object)

In [14]:
resumen_rqls_por_marca_week_copy = resumen_rqls_por_marca_week.copy()

test = resumen_rqls_por_marca_week_copy.drop(columns=["state", "day"])

In [15]:
test_sum_rql_per_brand =test.groupby(["brand", "Semana", "Año"])["rql"].sum().reset_index()
test_sum_rql_per_brand

,brand,Semana,Año,rql
0,Bajaj,14,2025,1592.0
1,Benelli,14,2025,66.0
2,CF Moto,14,2025,433.0
3,CFLITE,14,2025,106.0
4,Dinamo,14,2025,23.0
5,Hero,14,2025,45.0
6,Honda,14,2025,291.0
7,Husqvarna,14,2025,6.0
8,Italika,14,2025,283.0
9,Jiajue,14,2025,3.0


In [16]:
test_sum_rql_per_brand["rql"].sum()
print(f"Total de rql: {test_sum_rql_per_brand['rql'].sum()}")

Total de rql: 4027.0
